In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import operator
import time

In [62]:
train_data = pd.read_csv('hw1_datasets/q3/sms_train_features.csv')
train_labels = pd.read_csv('hw1_datasets/q3/sms_train_labels.csv')
test_data = pd.read_csv('hw1_datasets/q3/sms_test_features.csv')
test_labels = pd.read_csv('hw1_datasets/q3/sms_test_labels.csv')
vocab = pd.read_csv('hw1_datasets/q3/vocabulary.txt', header=None)

In [272]:
vocab

,0
0,point
1,crazy
2,available
3,great
4,world
...,...
3453,dental
3454,dump
3455,heap
3456,salesman


In [351]:
test_labels.tail()

,Unnamed: 0,class
973,973,1
974,974,0
975,975,0
976,976,0
977,977,1


In [352]:
test_data.tail()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3448,3449,3450,3451,3452,3453,3454,3455,3456,3457
973,973,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
974,974,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
975,975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
976,976,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
977,977,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
train_data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3449,3450,3451,3452,3453,3454,3455,3456,3457,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3906,3906,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3907,3907,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3908,3908,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3909,3909,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
train_labels.tail()

,Unnamed: 0,class
3906,3906,1
3907,3907,0
3908,3908,0
3909,3909,0
3910,3910,0


## Data Description
Training set: 3911 words, The j-th term in a row i is the occurrence information of the j-th vocabulary word in the i-th message.Vocab has 3458 words.
Label file has ground truths(gts), label 0:normal message, label 1:spam 

## Maximum Likelihood Estimation(MLE)

We will join train data and train ground truths in order to filter and count spam and ham messages easily

In [65]:
train_data = train_data.join(train_labels['class'])

ValueError: columns overlap but no suffix specified: Index(['class'], dtype='object')

In [66]:
train_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3449,3450,3451,3452,3453,3454,3455,3456,3457,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Filter the messages as if they only contain spam messages

In [291]:
train_data_spams = train_data[train_data['class'] == 1]
train_data_hams = train_data[train_data['class'] == 0]

In [334]:
train_data_spams

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3449,3450,3451,3452,3453,3454,3455,3456,3457,class
14,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27,27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
32,32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34,34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
48,48,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3895,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3898,3898,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3900,3900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3902,3902,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [340]:
tj_spam = np.zeros(3457)

for i in range(len(tj_spam)):
    tj_spam[i] = train_data_spams[str(i)].sum()

2.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
7.0
1.0
2.0
0.0
1.0
1.0
12.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
3.0
2.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
3.0
0.0
0.0
8.0
0.0
0.0
0.0
4.0
0.0
0.0
0.0
3.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
7.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
0.0
0.0
0.0
0.0
0.0
0.0
3.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
3.0
0.0
1.0
7.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
26.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.0
0.0
5.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
5.0
1.0
3.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.0
0.0
0.0
0.0
0.0
0.0
13.0
0.0
2.0
0.0
0.0
0.0
0.0
3.0
3.0
0.0
0.0
0.0
0.0
19.0
0.0
26.0
3.0
0.0
0.0
0.0
0.0
0.0
8.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
7.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.

1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
26.0
2.0
3.0
2.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
3.0
1.0
0.0
1.0
4.0
3.0
0.0
0.0
0.0
0.0
1.0
0.0
2.0
3.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
2.0
3.0
1.0
0.0
1.0
0.0
11.0
0.0
1.0
0.0
0.0
0.0
93.0
0.0
8.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
2.0
1.0
0.0
10.0
4.0
0.0
0.0
4.0
0.0
8.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
4.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
12.0
0.0
0.0
0.0
0.0
3.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.0
0.0
0.0
0.0
0.0
3.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
2.0
2.0
0.0
3.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
8.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
22.0
4.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
56.0
1.0
2.0
0.0
0.0
0.0
9.0
0.0
0.0
2

In [71]:
tj_ham = np.zeros(3457)
for i in range(len(tj_ham)):
    tj_ham[i] = train_data_hams[str(i)].sum()
tj_ham

array([11.,  1.,  1., ...,  4.,  3.,  1.])

In [87]:
n_normal = train_data_hams.shape[0] # number of the positive messages in the training set
n_total = train_data.shape[0] # total number of the messages in the training set
pi_y_normal = n_normal / n_total
print('Nnormal ' + str(n_normal) + '\n' 'N ' + str(n_total) + '\n' + 'πy=normal is ' + str(pi_y_normal))

Nnormal 3314
N 3911
πy=normal is 0.8473536180005113


In [108]:
prob_j_spam = tj_spam / np.sum(tj_spam)
print('θj|y=spam is: \n' + str(prob_j_spam))

θj|y=spam is: 
[0.00040584 0.00020292 0.         ... 0.         0.         0.        ]


In [109]:
prob_j_ham = tj_ham / np.sum(tj_ham)
print('θj|y=ham is: \n' + str(prob_j_ham))

θj|y=ham is: 
[6.21153086e-04 5.64684624e-05 5.64684624e-05 ... 2.25873849e-04
 1.69405387e-04 5.64684624e-05]


In [427]:
mle = np.zeros(test_data.shape)
test_range = test_data.shape[0]
print(mle)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [432]:
def summation_over_vocab_spam(i):
    sum_spam = 0.0
    for j in range(3457):
        if(prob_j_spam[j] != 0.0):
            sum_spam += float(test_data.iloc[i][j]) * math.log(prob_j_spam[j])
    return sum_spam

In [431]:
def summation_over_vocab_ham(i):
    sum_ham = 0.0
    for j in range(3457):
        if(prob_j_ham[j] != 0.0):
            sum_ham +=  float(test_data.iloc[i][j]) * math.log(prob_j_ham[j])
    return sum_ham 

In [434]:
# index 0 for the ham, index 1 for the spam
for i in range(test_range):
    mle[i][0] = math.log(pi_y_normal) + summation_over_vocab_ham(i)
    mle[i][1] = math.log(1 - pi_y_normal) + summation_over_vocab_spam(i)
    #print('sms training ' + str(i))

print('DONE')

DONE


In [435]:
mle

array([[  -69.05600083,   -10.38231977,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [  -31.50142741,   -17.49871391,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [  -75.04576162,   -41.21501996,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       ...,
       [-7233.57776056, -7616.1824228 ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [-7266.74090121, -7632.49465263,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [-7280.25606069, -7640.30419395,     0.        , ...,
            0.        ,     0.        ,     0.        ]])

In [442]:
estimations = np.zeros(test_range)
for i in range(test_range):
    if mle[i][0] >= mle[i][1]: # 0 likelihood greater than 1 likelihood
        estimations[i] = 0
    else:
        estimations[i] = 1
print(estimations.sum())

74.0


In [443]:
print(estimations.shape)

(978,)


In [444]:
print(test_labels['class'].shape)

(978,)


In [445]:
print(test_labels)

     Unnamed: 0  class
0             0      1
1             1      0
2             2      0
3             3      0
4             4      1
..          ...    ...
973         973      1
974         974      0
975         975      0
976         976      0
977         977      1

[978 rows x 2 columns]


# Accuracy & Validation

In [466]:
def calc_accuracy(gt_y, pred_y):
    correct = 0
    for g_y, p_y in zip(gt_y, pred_y):
        if g_y == p_y:
            correct += 1
    return (correct/float(len(gt_y)) * 100)

In [470]:
def confusion_matrix(gt_y, pred_y):
    # true positives
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for g_y, p_y in zip(gt_y, pred_y):
        if g_y == 0 and p_y == 0:
            tn += 1
        elif g_y == 1 and p_y == 1:
            tp += 1
        elif g_y == 1 and p_y == 0:
            fn += 1
        elif g_y == 0 and p_y == 1:
            fp += 1
    return tp,tn,fp,fn

In [486]:
start_time = time.time()
print('Accuracy is ' + str(calc_accuracy(test_labels['class'], estimations)))
print('\n')
confusion_m = confusion_matrix(test_labels['class'], estimations)
print('Confusion matrix: ' + 'true positives:' + str(confusion_m[0]) + ', true negatives:' + str(confusion_m[1]) + ', false positives:' + str(confusion_m[2]) + ', false negatives:' + str(confusion_m[3]))
print("--- %s seconds ---" % (time.time() - start_time))

Accuracy is 83.33333333333334


Confusion matrix: true positives:10, true negatives:805, false positives:33, false negatives:130
--- 0.0019762516021728516 seconds ---


# Bernoulli Naive Bayes Model

### Parameters

In [494]:
train_data = pd.read_csv('hw1_datasets/q3/sms_train_features.csv')
train_labels = pd.read_csv('hw1_datasets/q3/sms_train_labels.csv')
test_data = pd.read_csv('hw1_datasets/q3/sms_test_features.csv')
test_labels = pd.read_csv('hw1_datasets/q3/sms_test_labels.csv')
vocab = pd.read_csv('hw1_datasets/q3/vocabulary.txt', header=None)

In [495]:
train_data = train_data.join(train_labels['class'])

In [496]:
train_data.shape

(3911, 3460)

## Mutual Information Feature Selection

In [489]:
train_data_spams.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3449,3450,3451,3452,3453,3454,3455,3456,3457,class
14,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27,27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
32,32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34,34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
48,48,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [490]:
train_data_spams.shape

(597, 3460)

In [492]:
train_data_spams.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3449,3450,3451,3452,3453,3454,3455,3456,3457,class
14,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27,27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
32,32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34,34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
48,48,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [498]:
edge_length = train_data.shape[1] - 2

In [499]:
print(edge_length)

3458


## Parameters to Learn

In [502]:
s_j_spam = np.count_nonzero(train_data_spams, axis=0)
s_j_spam = np.delete(s_j_spam,0,0)
s_j_spam = np.delete(s_j_spam,edge_length,0)

In [501]:
s_j_ham = np.count_nonzero(train_data_hams, axis=0)
s_j_ham = np.delete(s_j_ham,0,0)
s_j_ham = np.delete(s_j_ham,edge_length,0)

In [404]:
n_normal = train_data_hams.shape[0] # number of the positive messages in the training set
n_total = train_data.shape[0] # total number of the messages in the training set
pi_y_normal = n_normal / n_total
print('Nnormal ' + str(n_normal) + '\n' 'N ' + str(n_total) + '\n' + 'πy=normal is ' + str(pi_y_normal))

Nnormal 3314
N 3911
πy=normal is 0.8473536180005113


In [413]:
theta_j_spam = s_j_spam / (1 - n_normal)
theta_j_ham = s_j_ham / n_normal
print(theta_j_spam)
print(theta_j_ham)

[-0.00060368 -0.00030184 -0.         ... -0.         -0.
 -0.        ]
[0.0030175  0.00030175 0.00030175 ... 0.00090525 0.00030175 0.00030175]


## MLE Function

In [459]:
def ber_summation_over_vocab_spam(i):
    sum_spam = 0.0
    for j in range(test_range):
        sum_spam += float(test_data.iloc[i][j] * theta_j_spam[i] + (1 - test_data.iloc[i][j]) * (1 - theta_j_spam[i] ))
    return sum_spam

In [458]:
def ber_summation_over_vocab_ham(i):
    sum_ham = 0.0
    for j in range(test_range):
        sum_ham += float(test_data.iloc[i][j] * theta_j_ham[i] + (1 - test_data.iloc[i][j]) * (1 - theta_j_ham[i]))
    return sum_ham 

In [460]:
mle_naive = np.zeros([test_data.shape[0],2])
test_range = test_data.shape[0]
print(mle_naive)
print(mle_naive[0])

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[0. 0.]


In [462]:
# index 0 for the ham, index 1 for the spam
for i in range(test_data.shape[0]):
    mle_naive[i][0] = math.log(pi_y_normal) + ber_summation_over_vocab_ham(i)
    mle_naive[i][1] = math.log(1 - pi_y_normal) + ber_summation_over_vocab_spam(i)
    #print('sms training ' + str(i))

sms training 0
sms training 1
sms training 2
sms training 3
sms training 4
sms training 5
sms training 6
sms training 7
sms training 8
sms training 9
sms training 10
sms training 11
sms training 12
sms training 13
sms training 14
sms training 15
sms training 16
sms training 17
sms training 18
sms training 19
sms training 20
sms training 21
sms training 22
sms training 23
sms training 24
sms training 25
sms training 26
sms training 27
sms training 28
sms training 29
sms training 30
sms training 31
sms training 32
sms training 33
sms training 34
sms training 35
sms training 36
sms training 37
sms training 38
sms training 39
sms training 40
sms training 41
sms training 42
sms training 43
sms training 44
sms training 45
sms training 46
sms training 47
sms training 48
sms training 49
sms training 50
sms training 51
sms training 52
sms training 53
sms training 54
sms training 55
sms training 56
sms training 57
sms training 58
sms training 59
sms training 60
sms training 61
sms training 62
sm

sms training 489
sms training 490
sms training 491
sms training 492
sms training 493
sms training 494
sms training 495
sms training 496
sms training 497
sms training 498
sms training 499
sms training 500
sms training 501
sms training 502
sms training 503
sms training 504
sms training 505
sms training 506
sms training 507
sms training 508
sms training 509
sms training 510
sms training 511
sms training 512
sms training 513
sms training 514
sms training 515
sms training 516
sms training 517
sms training 518
sms training 519
sms training 520
sms training 521
sms training 522
sms training 523
sms training 524
sms training 525
sms training 526
sms training 527
sms training 528
sms training 529
sms training 530
sms training 531
sms training 532
sms training 533
sms training 534
sms training 535
sms training 536
sms training 537
sms training 538
sms training 539
sms training 540
sms training 541
sms training 542
sms training 543
sms training 544
sms training 545
sms training 546
sms training 5

sms training 971
sms training 972
sms training 973
sms training 974
sms training 975
sms training 976
sms training 977


In [503]:
print(mle_naive)

[[ 9.74883246e+02  9.76710770e+02]
 [ 9.75540458e+02  9.74414362e+02]
 [ 9.72542269e+02  9.71120369e+02]
 ...
 [ 1.42337912e+00 -8.79631261e-01]
 [ 1.29474471e-01 -1.87963126e+00]
 [-8.69922029e-01 -2.87963126e+00]]


### Estimating the probabilities

In [504]:
estimations_naive = np.zeros(test_range)
for i in range(test_range):
    if mle_naive[i][0] >= mle_naive[i][1]: # 0 likelihood greater than 1 likelihood
        estimations[i] = 0
    else:
        estimations[i] = 1
print(estimations.sum())

43.0


### Accuracy and Confusion Matrix

In [484]:
start_time = time.time()
print('NAIVE BAYES')
print('Accuracy is:' + str(calc_accuracy(test_labels['class'], estimations)))
print('\n')
confusion_m = confusion_matrix(test_labels['class'], estimations)
print('Confusion matrix: ' + 'true positives:' + str(confusion_m[0]) + ', true negatives:' + str(confusion_m[1]) + ', false positives:' + str(confusion_m[2]) + ', false negatives:' + str(confusion_m[3]))
print("--- %s seconds ---" % (time.time() - start_time))

NAIVE BAYES
Accuracy is:83.33333333333334


Confusion matrix: true positives:10, true negatives:805, false positives:33, false negatives:130
--- 0.0020551681518554688 seconds ---
